In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!pip install greykite==0.5.1 prophet==1.1.2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 16.4 MB/s eta 0:00:00


In [3]:
# from greykite.framework.templates.autogen.forecast_config import ForecastConfig
# from greykite.framework.templates.autogen.forecast_config import MetadataParam
# from greykite.framework.templates.forecaster import Forecaster
# from greykite.framework.templates.model_templates import ModelTemplateEnum
# from greykite.framework.utils.result_summary import summarize_grid_search_results

In [4]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

from prophet import Prophet

import xgboost as xgb

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import pacf

from scipy.stats import skew, kurtosis
from statsmodels.tsa.seasonal import seasonal_decompose


import numpy as np
import pandas as pd
import pmdarima as pm

In [12]:
def calculate_mape(true_values, predicted_values):
    epsilon = 1e-10
    abs_percentage_error = np.abs((true_values - predicted_values) / (true_values + epsilon))
    return round(np.mean(abs_percentage_error) * 100 , 4)


def forecast_autoarima(df, forecast_period=0):
    model = pm.auto_arima(df["value"].tolist(), seasonal=True)
    past = model.predict_in_sample().tolist()
    if forecast_period:
        forecasts = model.predict(forecast_period).tolist()
        past.extend(forecasts)
        return past
    return past

# def forecast_greykite(df , freq ="D" ,forecast=False , forecast_period = 100):
#   metadata = MetadataParam(
#      time_col="date",
#      value_col="value",
#      freq=freq ,
#      train_end_date=df['date'].iloc[-1]
#   )
#   forecaster = Forecaster()
#   result = forecaster.run_forecast_config(
#       df=df,
#       config=ForecastConfig(
#           model_template=ModelTemplateEnum.SILVERKITE.name,
#           forecast_horizon=forecast_period,
#           coverage=0.95,
#           metadata_param=metadata
#       )
#   )

#   if not forecast:
#     return result.forecast.df['forecast'].tolist()[:-forecast_period]
#   return result.forecast.df['forecast'].tolist()

def forecast_xgboost(df , forecast_period = 0 , date_to = None):
  df['year'] = df['date'].dt.year
  df['month'] = df['date'].dt.month
  df['day'] = df['date'].dt.day
  df['day_of_week'] = df['date'].dt.dayofweek

  X_train = df.drop(['value' , 'date'], axis=1)
  y_train = df['value']

  model = xgb.XGBRegressor(max_depth=3, min_child_weight=1, reg_alpha=0.1)
  model.fit(X_train, y_train)

  past = model.predict(X_train).tolist()

  if forecast_period and date_to:
    date_range = pd.date_range(start=date_to + pd.DateOffset(days=1), periods=forecast_period, freq='D' , normalize=True)
    future_data = pd.DataFrame(index=date_range)
    future_data['year'] = future_data.index.year
    future_data['month'] = future_data.index.month
    future_data['day'] = future_data.index.day
    future_data['day_of_week'] = future_data.index.dayofweek

    future_predictions = model.predict(future_data).tolist()
    past.extend(future_predictions)
    return past

  return past


def forecast_ets(df , forecast_period = 0 ):
  try:
      model = ExponentialSmoothing(
          df["value"], trend="add", seasonal="add", seasonal_periods=12
      )
  except:
      model = ExponentialSmoothing(df["value"])

  result = model.fit()

  past = result.fittedvalues.tolist()
  if forecast_period:
    future_data = result.forecast(steps = forecast_period)
    past.extend(future_data)
    return past

  return past

# def forecast_neural_prophet(df , forecast_period = 0 ):
#   df = df.rename(columns={'date': 'ds', 'value': 'y'})

#   m = NeuralProphet()
#   m.fit(df)

#   future = m.make_future_dataframe(periods=forecast_period)
#   forecast = m.predict(future)
#   return forecast['yhat'].tolist()

def forecast_prophet(df , forecast_period = 0 ):
  df = df.rename(columns={'date': 'ds', 'value': 'y'})

  model = Prophet()
  model.fit(df)

  future = model.make_future_dataframe(periods=forecast_period)
  forecast = model.predict(future)
  return forecast['yhat'].tolist()


def forecast_arima(df , forecast_period = 0):
  result = adfuller(df['value'])
  p_value = result[1]

  if p_value < 0.05:
      d = 0
  else:
      d = 1

  pacf_values = pacf(df['value'], method='ols')
  acf_values = pacf(df['value'], method='ols')

  for p in range(1, len(pacf_values)):
      if abs(pacf_values[p]) < 1.96 / np.sqrt(len(df)):
          break

  for q in range(1, len(acf_values)):
      if abs(acf_values[q]) < 1.96 / np.sqrt(len(df)):
          break

  model = sm.tsa.arima.ARIMA(df['value'], order=(p, d, q))
  results = model.fit()
  past = results.fittedvalues.tolist()

  if forecast_period:
    future_data = results.forecast(steps=forecast_period)
    past.extend(future_data)
    return past

  return past

# def train_greykite(df, freq="D"):
#     metadata = MetadataParam(
#         time_col="date",
#         value_col="value",
#         freq=freq,
#         train_end_date=df['date'].iloc[-1]
#     )
#     forecaster = Forecaster()
#     result = forecaster.run_forecast_config(
#         df=df,
#         config=ForecastConfig(
#             model_template=ModelTemplateEnum.SILVERKITE.name,
#             metadata_param=metadata
#         )
#     )

#     trained_values  = result.forecast.df['forecast'].tolist()[:len(df)]
#     return trained_values

In [6]:
 file_path = '/content/drive/MyDrive/DGdataset/Sample Time Series/daily/processed_daily.csv'
df = pd.read_csv(file_path , index_col = 0)

df['date'] = pd.to_datetime(df['date'])
df.set_index(df['date'], inplace=True)
df.sort_index(inplace=True)
df.rename_axis('index',inplace=True)
df.head()

,date,value
index,,
2021-02-01,2021-02-01,903053.004725
2021-02-02,2021-02-02,910297.753775
2021-02-03,2021-02-03,912559.254575
2021-02-04,2021-02-04,920697.253775
2021-02-05,2021-02-05,927060.753775


In [7]:
df.tail()

,date,value
index,,
2023-01-28,2023-01-28,27.0
2023-01-29,2023-01-29,153181.0
2023-01-30,2023-01-30,541232.0
2023-01-31,2023-01-31,712411.0
2023-02-01,2023-02-01,1.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 731 entries, 2021-02-01 to 2023-02-01
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    731 non-null    datetime64[ns]
 1   value   731 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 17.1 KB


In [ ]:
import warnings
import logging
import sys


warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning, module="prophet")

logging.basicConfig(level=logging.WARNING)

# stdout_orig = sys.stdout
# stderr_orig = sys.stderr

# sys.stdout = open("output.log", "w")  # Replace "output.log" with the desired file name
# sys.stderr = open("error.log", "w")    # Replace "error.log" with the desired file name

In [ ]:
def generate_data(df , sample_size = 100):

  results_df = pd.DataFrame(columns =[
      'Sample Index',
      'Mean',
      'Variance',
      'Skewness',
      'Kurtosis',
      'Trend Mean',
      'Seasonal Mean',
      'Residual Mean',
      'Best Model',
  ])
  track_log = {}

  #(len(df)-sample_size)
  for start_index in range(len(df)-sample_size):
    sample_df = df.iloc[start_index:start_index+sample_size]

    true_values = sample_df['value']

    mape_prophet = calculate_mape(true_values, forecast_prophet(sample_df))
    mape_arima = calculate_mape(true_values, forecast_arima(sample_df))
    mape_ets = calculate_mape(true_values, forecast_ets(sample_df))
    mape_xgboost = calculate_mape(true_values, forecast_xgboost(sample_df))
    # mape_greykite = calculate_mape(true_values, train_greykite(sample_df))
    # mape_xgboost = float("inf")
    mape_greykite = float("inf")

    res = [
        ('Prophet', mape_prophet),
        ('ARIMA', mape_arima),
        ('ETS', mape_ets),
        ('XGBoost', mape_xgboost),
        ('Greykite', mape_greykite)
    ]

    track_log[start_index] = res
    best_model = min(res, key=lambda x: x[1])[0]

    # Calculate statistical features
    mean = sample_df['value'].mean()
    variance = sample_df['value'].var()
    skewness = skew(sample_df['value'])
    kurt = kurtosis(sample_df['value'])

    # Calculate time-based features
    result = seasonal_decompose(sample_df['value'], model='additive', extrapolate_trend='freq')
    trend_mean = result.trend.mean()
    seasonal_mean = result.seasonal.mean()
    residual_mean = result.resid.mean()

    result_dict = {
          'Sample Index': start_index,
          'Mean': mean,
          'Variance': variance,
          'Skewness': skewness,
          'Kurtosis': kurt,
          'Trend Mean': trend_mean,
          'Seasonal Mean': seasonal_mean,
          'Residual Mean': residual_mean,
          'Best Model': best_model,
      }

    results_df = results_df.append(result_dict, ignore_index=True)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++" , start_index)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++" , result_dict)


  return track_log , results_df

In [10]:
from sklearn.model_selection import TimeSeriesSplit

n_splits = 200
tscv = TimeSeriesSplit(n_splits=n_splits)

X = df.drop(['value'], axis=1)
y = df['value']

results_df = pd.DataFrame(columns =[
    'Sample Index',
    'Mean',
    'Variance',
    'Skewness',
    'Kurtosis',
    'Trend Mean',
    'Seasonal Mean',
    'Residual Mean',
    'Best Model',
])
track_log = {}

for start_index, (train_index, test_index) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    sample_df = pd.concat([X_train, y_train], axis=1)


    mape_prophet = calculate_mape(y_test, forecast_prophet(sample_df , forecast_period = len(X_test))[-len(X_test):] )
    mape_auto_arima = calculate_mape(y_test, forecast_autoarima(sample_df, forecast_period = len(X_test))[-len(X_test):])
    mape_ets = calculate_mape(y_test, forecast_ets(sample_df , forecast_period = len(X_test))[-len(X_test):])
    mape_xgboost = calculate_mape(y_test, forecast_xgboost(sample_df ,  forecast_period = len(X_test) , date_to = X_test['date'][0])[-len(X_test):])


    res = [
        ('Prophet', mape_prophet),
        ('ARIMA', mape_auto_arima),
        ('ETS', mape_ets),
        ('XGBoost', mape_xgboost),
    ]

    track_log[start_index] = res
    best_model = min(res, key=lambda x: x[1])[0]

    # Calculate statistical features
    mean = sample_df['value'].mean()
    variance = sample_df['value'].var()
    skewness = skew(sample_df['value'])
    kurt = kurtosis(sample_df['value'])

    # Calculate time-based features
    result = seasonal_decompose(sample_df['value'], model='additive', extrapolate_trend='freq')
    trend_mean = result.trend.mean()
    seasonal_mean = result.seasonal.mean()
    residual_mean = result.resid.mean()

    result_dict = {
          'Sample Index': start_index,
          'Mean': mean,
          'Variance': variance,
          'Skewness': skewness,
          'Kurtosis': kurt,
          'Trend Mean': trend_mean,
          'Seasonal Mean': seasonal_mean,
          'Residual Mean': residual_mean,
          'Best Model': best_model,
      }

    results_df = results_df.append(result_dict, ignore_index=True)

    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++" , res)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++" , best_model , start_index , len(X_train) ,len(X_test))

results_df.to_csv('/content/drive/MyDrive/DGdataset/Sample Time Series/daily/classification_attempt_6.csv')

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnc3vi0u7/y4qp6ea_.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnc3vi0u7/xgf00nt_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=47387', 'data', 'file=/tmp/tmpnc3vi0u7/y4qp6ea_.json', 'init=/tmp/tmpnc3vi0u7/xgf00nt_.json', 'output', 'file=/tmp/tmpnc3vi0u7/prophet_modelmvlkul83/prophet_model-20231001045702.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:57:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:57:02 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.10/dist-packages/

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 7.6855), ('ARIMA', 10.5272), ('ETS', 11.9049), ('XGBoost', 9.1174)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ Prophet 0 131 3


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
<ipython-input-10-fd7e9e95ee04>:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnc3vi0u7/ev3gb4rz.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnc3vi0u7/lew19wzp.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_thread

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 33.8809), ('ARIMA', 25.0045), ('ETS', 47.0336), ('XGBoost', 25.0435)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ ARIMA 1 134 3


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
<ipython-input-10-fd7e9e95ee04>:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnc3vi0u7/2v0teg_v.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnc3vi0u7/3z30tagf.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_thread

...

05:54:13 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
<ipython-input-10-fd7e9e95ee04>:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnc3vi0u7/2d4fklmz.json
DEBUG:cmdstanpy:input tempfile:

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 345473.6148), ('ARIMA', 337974.8431), ('ETS', 506102.0268), ('XGBoost', 252254.8934)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ XGBoost 198 725 3


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 8937577.7878), ('ARIMA', 11086053.4454), ('ETS', 8878404.2548), ('XGBoost', 9604409.7159)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ ETS 199 728 3


<ipython-input-10-fd7e9e95ee04>:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)


In [11]:
results_df.head()

,Sample Index,Mean,Variance,Skewness,Kurtosis,Trend Mean,Seasonal Mean,Residual Mean,Best Model
0,0,1.170354e+06,4.007895e+10,1.398559,3.071583,1.170865e+06,-3.686601e+03,3176.103442,Prophet
1,1,1.173173e+06,4.021102e+10,1.345245,2.854922,1.173717e+06,-4.948876e+02,-48.892901,ARIMA
2,2,1.165945e+06,4.167459e+10,1.259848,2.653146,1.170509e+06,-2.861151e+03,-1702.486047,ETS
3,3,1.161884e+06,4.173290e+10,1.257650,2.661183,1.162507e+06,1.455192e-11,-623.829597,XGBoost
4,4,1.154895e+06,4.314659e+10,1.184845,2.477578,1.153841e+06,-1.812498e+03,2866.969396,ARIMA


In [12]:
track_log

{0: [('Prophet', 7.6855),
  ('ARIMA', 10.5272),
  ('ETS', 11.9049),
  ('XGBoost', 9.1174)],
 1: [('Prophet', 33.8809),
  ('ARIMA', 25.0045),
  ('ETS', 47.0336),
  ('XGBoost', 25.0435)],
 2: [('Prophet', 32.8295),
  ('ARIMA', 25.4999),
  ('ETS', 22.4939),
  ('XGBoost', 27.6526)],
 3: [('Prophet', 20.0868),
  ('ARIMA', 20.7693),
  ('ETS', 28.5855),
  ('XGBoost', 13.2791)],
 4: [('Prophet', 14.0931),
  ('ARIMA', 4.0952),
  ('ETS', 18.3795),
  ('XGBoost', 15.3302)],
 5: [('Prophet', 8.134),
  ('ARIMA', 6.2477),
  ('ETS', 9.5807),
  ('XGBoost', 13.2208)],
 6: [('Prophet', 6.9967),
  ('ARIMA', 1.2088),
  ('ETS', 2.8798),
  ('XGBoost', 42.7734)],
 7: [('Prophet', 11.6679),
  ('ARIMA', 9.0425),
  ('ETS', 22.9558),
  ('XGBoost', 18.67)],
 8: [('Prophet', 7.5939),
  ('ARIMA', 19.5503),
  ('ETS', 18.0956),
  ('XGBoost', 38.984)],
 9: [('Prophet', 17.874),
  ('ARIMA', 15.4918),
  ('ETS', 9.7231),
  ('XGBoost', 21.2661)],
 10: [('Prophet', 6.1749),
  ('ARIMA', 10.299),
  ('ETS', 6.3276),
  ('XGBoos

In [14]:
results_df['Best Model'].value_counts()

ARIMA      140
XGBoost     35
ETS         14
Prophet     11
Name: Best Model, dtype: int64

# **Data Generation for rest of the data for classification Model**

In [7]:
from sklearn.model_selection import TimeSeriesSplit

In [14]:
folder_path = ["/content/drive/MyDrive/DGdataset/Sample Time Series/weekly/processed_weekly.csv" ]
temp = []

for index , folder in enumerate(folder_path):

  df = pd.read_csv(folder)
  print(df.info())
  df['date'] = pd.to_datetime(df['date'])
  df.set_index(df['date'], inplace=True)
  df.sort_index(inplace=True)
  df.rename_axis('index',inplace=True)


  n_splits = 75

  tscv = TimeSeriesSplit(n_splits=n_splits)

  X = df.drop(['value'], axis=1)
  y = df['value']

  results_df = pd.DataFrame(columns =[
      'Sample Index',
      'Mean',
      'Variance',
      'Skewness',
      'Kurtosis',
      'Trend Mean',
      'Seasonal Mean',
      'Residual Mean',
      'Best Model',
  ])
  track_log = {}

  for start_index, (train_index, test_index) in enumerate(tscv.split(X)):
      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      y_train, y_test = y[train_index], y[test_index]

      sample_df = pd.concat([X_train, y_train], axis=1)


      mape_prophet = calculate_mape(y_test, forecast_prophet(sample_df , forecast_period = len(X_test))[-len(X_test):] )
      mape_auto_arima = calculate_mape(y_test, forecast_autoarima(sample_df, forecast_period = len(X_test))[-len(X_test):])
      mape_ets = calculate_mape(y_test, forecast_ets(sample_df , forecast_period = len(X_test))[-len(X_test):])
      mape_xgboost = calculate_mape(y_test, forecast_xgboost(sample_df ,  forecast_period = len(X_test) , date_to = X_test['date'][0])[-len(X_test):])


      res = [
          ('Prophet', mape_prophet),
          ('ARIMA', mape_auto_arima),
          ('ETS', mape_ets),
          ('XGBoost', mape_xgboost),
      ]

      track_log[start_index] = res
      best_model = min(res, key=lambda x: x[1])[0]

      # Calculate statistical features
      mean = sample_df['value'].mean()
      variance = sample_df['value'].var()
      skewness = skew(sample_df['value'])
      kurt = kurtosis(sample_df['value'])

      # Calculate time-based features
      try:
          result = seasonal_decompose(
              sample_df["value"], model="additive", extrapolate_trend="freq"
          )
          trend_mean = result.trend.mean()
          seasonal_mean = result.seasonal.mean()
          residual_mean = result.resid.mean()
      except:
          result = seasonal_decompose(
              sample_df["value"], model="additive", extrapolate_trend="freq", period=1
          )
          trend_mean = result.trend.mean()
          seasonal_mean = result.seasonal.mean()
          residual_mean = result.resid.mean()


      result_dict = {
            'Sample Index': start_index,
            'Mean': mean,
            'Variance': variance,
            'Skewness': skewness,
            'Kurtosis': kurt,
            'Trend Mean': trend_mean,
            'Seasonal Mean': seasonal_mean,
            'Residual Mean': residual_mean,
            'Best Model': best_model,
        }

      results_df = results_df.append(result_dict, ignore_index=True)

      print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++" , res)
      print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++" , best_model , start_index , len(X_train) ,len(X_test))

  temp.append(track_log)
  if "hourly" in folder:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++hourly")
    print(results_df['Best Model'].value_counts())

    results_df.to_csv('/content/drive/MyDrive/DGdataset/Sample Time Series/hourly/classification_attempt_1.csv')

  if "weekly" in folder:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++weekly")
    print(results_df['Best Model'].value_counts())

    results_df.to_csv('/content/drive/MyDrive/DGdataset/Sample Time Series/weekly/classification_attempt_1.csv')

  if "monthly" in folder:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++monthly")
    print(results_df['Best Model'].value_counts())

    results_df.to_csv('/content/drive/MyDrive/DGdataset/Sample Time Series/monthly/classification_attempt_1.csv')



INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 2.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    79 non-null     object 
 1   value   79 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.4+ KB
None


DEBUG:cmdstanpy:input tempfile: /tmp/tmpz20i81ve/dxg3_bip.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpz20i81ve/79lpz4ai.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=90773', 'data', 'file=/tmp/tmpz20i81ve/dxg3_bip.json', 'init=/tmp/tmpz20i81ve/79lpz4ai.json', 'output', 'file=/tmp/tmpz20i81ve/prophet_modelo5omjvh5/prophet_model-20231001111742.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
11:17:42 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:17:42 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-pack

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 11.0062), ('ARIMA', 9.3276), ('ETS', 9.0695), ('XGBoost', 8.7394)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ XGBoost 0 4 1


11:17:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
<ipython-input-14-94b3a4e46828>:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to

...

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 136776.7548), ('ARIMA', 19962.3462), ('ETS', 37854.8527), ('XGBoost', 65308.1178)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ ARIMA 72 76 1


11:19:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
<ipython-input-14-94b3a4e46828>:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to ove

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 207124.7999), ('ARIMA', 26959.2745), ('ETS', 17949.2408), ('XGBoost', 128850.0681)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ ETS 73 77 1


11:19:11 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
<ipython-input-14-94b3a4e46828>:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 126711.9792), ('ARIMA', 14971.1109), ('ETS', 10930.1056), ('XGBoost', 30705.1276)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ ETS 74 78 1
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++weekly
XGBoost    27
Prophet    19
ETS        16
ARIMA      13
Name: Best Model, dtype: int64


In [15]:
temp

[{0: [('Prophet', 11.0062),
   ('ARIMA', 9.3276),
   ('ETS', 9.0695),
   ('XGBoost', 8.7394)],
  1: [('Prophet', 4.8666),
   ('ARIMA', 2.2163),
   ('ETS', 4.134),
   ('XGBoost', 0.054)],
  2: [('Prophet', 1.1517),
   ('ARIMA', 0.1028),
   ('ETS', 1.5376),
   ('XGBoost', 3.2329)],
  3: [('Prophet', 32.3899),
   ('ARIMA', 31.8824),
   ('ETS', 32.5961),
   ('XGBoost', 31.3423)],
  4: [('Prophet', 45.0684),
   ('ARIMA', 51.3381),
   ('ETS', 37.56),
   ('XGBoost', 33.3411)],
  5: [('Prophet', 23.2615),
   ('ARIMA', 14.7365),
   ('ETS', 4.2008),
   ('XGBoost', 4.4068)],
  6: [('Prophet', 14.8259),
   ('ARIMA', 26.3541),
   ('ETS', 30.0793),
   ('XGBoost', 29.9187)],
  7: [('Prophet', 2.805),
   ('ARIMA', 40.4424),
   ('ETS', 10.8869),
   ('XGBoost', 10.9511)],
  8: [('Prophet', 18.9498),
   ('ARIMA', 19.0654),
   ('ETS', 23.7972),
   ('XGBoost', 7.9198)],
  9: [('Prophet', 10.6468),
   ('ARIMA', 0.2183),
   ('ETS', 0.3371),
   ('XGBoost', 2.8034)],
  10: [('Prophet', 9.7758),
   ('ARIMA', 5.